# SQL kunskapskontroll exemple

You should upload the notebook with all the cells run, so the results are visible

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from sqlalchemy import create_engine

In [16]:
engine = create_engine('mssql://DESKTOP-CIKOHBH/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
connection = engine.connect()

# 1. SQL code used at data analysis

In [17]:
# Write about the why and not the how (the how is in the code)
query_1 = """
SELECT 
  P.BusinessEntityID, 
  P.FirstName, 
  P.LastName, 
  HR.JobTitle,
  EmpPay.Rate
FROM HumanResources.Employee AS HR 
INNER JOIN Person.Person AS P 
    ON HR.BusinessEntityID = P.BusinessEntityID
INNER JOIN HumanResources.EmployeePayHistory AS EmpPay
    ON EmpPay.BusinessEntityID = HR.BusinessEntityID
WHERE 
  Gender = 'M'
ORDER BY P.BusinessEntityID DESC;
"""
pd.read_sql(query_1, con=connection)

,BusinessEntityID,FirstName,LastName,JobTitle,Rate
0,290,Ranjit,Varkey Chudukatil,Sales Representative,23.0769
1,285,Syed,Abbas,Pacific Sales Manager,48.1010
2,284,Tete,Mensa-Annan,Sales Representative,23.0769
3,283,David,Campbell,Sales Representative,23.0769
4,282,José,Saraiva,Sales Representative,23.0769
...,...,...,...,...,...
223,4,Rob,Walters,Senior Tool Designer,8.6200
224,4,Rob,Walters,Senior Tool Designer,23.7200
225,4,Rob,Walters,Senior Tool Designer,29.8462
226,3,Roberto,Tamburello,Engineering Manager,43.2692


# 2. Python code used at data analysis

In [18]:
# Write about the why and not the how (the how is in the code)
df = pd.read_sql(query_1, con=connection)
df

,BusinessEntityID,FirstName,LastName,JobTitle,Rate
0,290,Ranjit,Varkey Chudukatil,Sales Representative,23.0769
1,285,Syed,Abbas,Pacific Sales Manager,48.1010
2,284,Tete,Mensa-Annan,Sales Representative,23.0769
3,283,David,Campbell,Sales Representative,23.0769
4,282,José,Saraiva,Sales Representative,23.0769
...,...,...,...,...,...
223,4,Rob,Walters,Senior Tool Designer,8.6200
224,4,Rob,Walters,Senior Tool Designer,23.7200
225,4,Rob,Walters,Senior Tool Designer,29.8462
226,3,Roberto,Tamburello,Engineering Manager,43.2692


### Exemple for confidence interval

### NOTE: this is a CODE example for the CI calculation, the data it contains is of secondary importance (we know ALL the order sales totals → we can calculate the population parameter exactly → no real need for CI) 

In [19]:
df = pd.read_sql(sql="SELECT * FROM Sales.SalesOrderHeader", con=connection)
df.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,None,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07
1,43660,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
2,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,...,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
3,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,...,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
4,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,...,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07


In [20]:
order_amounts = df.SubTotal

In [21]:
# Calculate confidence interval for the average order amount
ci = stats.norm.interval(0.95, loc=np.mean(order_amounts), scale=stats.sem(order_amounts))

print(f"Confidence Interval for Average Order Amount: {ci}")

Confidence Interval for Average Order Amount: (3368.4909410451914, 3613.6404051426357)
